In [ ]:
from aocd import get_data
inp = get_data(day=8, year=2024)

Each antenna is tuned to a specific frequency indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas.

The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency a, they create the two antinodes marked with #:

```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```

Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```

Antennas with different frequencies don't create antinodes; A and a count as different frequencies. However, antinodes can occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital A creates no antinodes but has a lowercase-a-frequency antinode at its location

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost A-frequency antenna:

```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```

Because the topmost A-frequency antenna overlaps with a 0-frequency antinode, there are 14 total unique locations that contain an antinode within the bounds of the map.

In [ ]:
sample="""............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""

Complicated problem statement.

Here's my understanding.

- Given a set of antennas of the "same frequency" (which means the same character)
- One needs to find all the "antinodes" which are at  map locations which:
   - (1) are exactly on the line connecting one pair of antennas
   - (2) are exactly 2x as far form one antenna as from the other antenna

- An antinode and an antenna may show a location.

- Goal is to count not the antinodes, but the unique locations oer all antinodes.

let us simplify the problem and consider how to analyze it with just two antenna.

Also, this might be more easily considered as a math problem to solve, rather than a programming problem to search and check.

- two points in 2d space, A and B.
- They define a line. 
- without loss of generality, define the unit vector pointing from A toward B. (e.g, use upper leftmost as the src)
- no we may define any point p on the line as that unit vector times a scalar alpha.
- The distance is defined by the equation abs(p - A) = 2 abs(p - B) for instance

Q: Why does this not define antinodes in between the antennas? 

The example only shows them not in between the antennas.

Can we use Euclidean distance with rounding to integral units to give final positions?

In [ ]:
import math
from dataclasses import dataclass

@dataclass
class Point:
    x: float
    y: float
    def times(self,alpha): return Point(self.x * alpha, self.y * alpha)
    def plus(self,other:'Point'): return Point(self.x+other.x,self.y+other.y)
    def minus(self,other:'Point'): return self.plus( other.times( -1.0 ))
    def length(self): return math.sqrt(self.x ** 2 + self.y ** 2)
    def distance_from(self,other:'Point'): return (self.plus(other.times(-1.0))).length()
    def __hash__(self): return hash( (self.x,self.y) )
    def __eq__(self,other:'Point'): return self.x == other.x and self.y == other.y

In [ ]:
Point(10,2).minus(Point(3,1))

Point(x=7.0, y=1.0)

In [ ]:
Point(0,0).distance_from(Point(3,4))

5.0

In [ ]:
Point(1,1).length()

1.4142135623730951

In [ ]:
Point(0,0).distance_from(Point(3,4))

5.0

In [ ]:
def antinodes_for_(a,b):
    ba = b.times(2).minus(a)
    ab = a.times(2).minus(b)
    return (ab,ba)

In [ ]:
antinodes_for_(Point(0,0),Point(1,0))

(Point(x=-1.0, y=0.0), Point(x=2.0, y=0.0))

In [ ]:
def all_pairs(points:list[Point]):
    for i,p in enumerate(points):
        for j,q in enumerate(points):
            if i<j: yield (p,q)

In [ ]:
def antinodes_for_points(points:list[Point]):
    for (p,q) in all_pairs(points):
        (pq,qp) = antinodes_for_(p,q)
        yield pq
        yield qp

In [ ]:
list(antinodes_for_points([Point(0,0), Point(1,0)]))

[Point(x=-1.0, y=0.0), Point(x=2.0, y=0.0)]

In [ ]:
list(antinodes_for_points([Point(0,0), Point(3,0), Point(3,4) ]))

[Point(x=-3.0, y=0.0),
 Point(x=6.0, y=0.0),
 Point(x=-3.0, y=-4.0),
 Point(x=6.0, y=8.0),
 Point(x=3.0, y=-4.0),
 Point(x=3.0, y=8.0)]

In [ ]:
def all_antinodes(d:dict[str,list[Point]]):
    antinodes = []
    for k,points in d.items():
        antinodes.extend(antinodes_for_points(points))
    return antinodes

In [ ]:
def parse_map(s:str) -> dict[str,list[Point]]:
    retval = dict()
    lines = s.strip().splitlines()
    for y,line in enumerate(lines):
        for x,ch in enumerate(line):
            if ch != '.':
                retval.setdefault(ch,[]).append(Point(x,y))
    return retval

In [ ]:
parse_map(sample)

{'0': [Point(x=8, y=1), Point(x=5, y=2), Point(x=7, y=3), Point(x=4, y=4)],
 'A': [Point(x=6, y=5), Point(x=8, y=8), Point(x=9, y=9)]}

In [ ]:
parse_map(sample)

{'0': [Point(x=8, y=1), Point(x=5, y=2), Point(x=7, y=3), Point(x=4, y=4)],
 'A': [Point(x=6, y=5), Point(x=8, y=8), Point(x=9, y=9)]}

In [ ]:
all_antinodes( parse_map(sample))

[Point(x=11.0, y=0.0),
 Point(x=2.0, y=3.0),
 Point(x=9.0, y=-1.0),
 Point(x=6.0, y=5.0),
 Point(x=12.0, y=-2.0),
 Point(x=0.0, y=7.0),
 Point(x=3.0, y=1.0),
 Point(x=9.0, y=4.0),
 Point(x=6.0, y=0.0),
 Point(x=3.0, y=6.0),
 Point(x=10.0, y=2.0),
 Point(x=1.0, y=5.0),
 Point(x=4.0, y=2.0),
 Point(x=10.0, y=11.0),
 Point(x=3.0, y=1.0),
 Point(x=12.0, y=13.0),
 Point(x=7.0, y=7.0),
 Point(x=10.0, y=10.0)]

**AI Prompt**

If I want to use the set() function in Python to unique a set of objects, what attributes or functions do those objects need to support set() to do its thing?

**AI Response**

For objects to work with Python's set(), they need to be hashable, which means they must:

1. Have a working __hash__() method
2. Have a working __eq__() method for equality comparison

The hash value should remain constant during the object's lifetime, and objects that compare equal should have the same hash value.

Would you like me to explain how to make your Point class hashable?

**AI Prompt**

Are lists or tuples already hashable?

**AI Response**

Tuples are hashable (as long as all their contents are also hashable), but lists are not hashable. This is because tuples are immutable while lists can be modified after creation.

Would you like me to explain why mutability affects hashability?

**AI Prompt**

Please just tell me the singature of the hash and eq methods which I need to implement

**AI Response**

Here are the method signatures:

```python
def __hash__(self) -> int
def __eq__(self, other) -> bool
```

Would you like me to explain what values these methods should return?

**AI Prompt**

Does my implementation of Point look like a valid implementation of hash and eq? I will treat Point as immutable btw.

**AI Response**

Looking at your Point class definition, there's a small bug in the __hash__ method. The current implementation uses undefined variables `x` and `y` instead of `self.x` and `self.y`. It should be:

```python
def __hash__(self): return hash((self.x, self.y))
```

The __eq__ method looks correct as implemented.

Would you like me to explain why using a tuple in the hash implementation is a good approach?

**AI Prompt**

Please list all the functions which I have defined so far, briefly.

**AI Response**

Looking at your code, you've defined:

1. Point class with methods:
   - times(alpha)
   - plus(other)
   - minus(other)
   - length()
   - distance_from(other)
   - __hash__()
   - __eq__(other)

2. Functions:
   - antinodes_for_(a,b)
   - all_pairs(points)
   - antinodes_for_points(points)
   - all_antinodes(d)
   - parse_map(s)

Would you like me to describe what any of these functions do?

In [ ]:
def count_antinodes_on_(mmap:str) -> int:
    lines = mmap.splitlines()
    lenY = len(lines)
    lenX = len(lines[0])
    def in_bounds(p:Point):
        x_ok = 0 <= p.x and p.x <= lenX - 1
        y_ok = 0 <= p.y and p.y <= lenY - 1
        return x_ok and y_ok 
    ns = all_antinodes( parse_map( mmap ))
    ns_set = set([n for n in ns if in_bounds(n)])
    return len(ns_set)

In [ ]:
count_antinodes_on_(sample)

14

In [ ]:
count_antinodes_on_(inp)

303

# Pt 2

After updating your model, it turns out that an antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```

In fact, the three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to 9.

The original example now has 34 antinodes, including the antinodes that appear on every antenna:

```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```

This  new stipulation seems to require us to take into account that there are finite grid positions with integral coordinates rather than handling coordinates as floats. Because if we counted all the points on a real line, they would be innumerable.

**AI Prompt**

Please refresh my memory on the functions i defined again please

**AI Response**

Here are all the functions you've defined:

1. Point class with methods:
   - times(alpha)
   - plus(other)
   - minus(other)
   - length()
   - distance_from(other)
   - __hash__()
   - __eq__(other)

2. Functions:
   - antinodes_for_(a,b)
   - all_pairs(points)
   - antinodes_for_points(points)
   - all_antinodes(d)
   - parse_map(s)
   - count_antinodes_on_(mmap)
   - in_bounds(p) (defined inside count_antinodes_on_)

Would you like me to explain what any of these do?

A plan: redefine `antinodes_for_(a,b)` to be aware of map dimensions, so that it can find all grid points.

Could compute where the connecting ray intersects the boundary and then use that to iterate

In [ ]:
def points_on_ray(a:Point,b:Point,mmap_corners):
    minP,maxP = mmap_corners
    max_length = minP.minus(maxP).length()
    uv = b.minus(a).times(1/(b.minus(a).length())) # half-unit vector
    def in_bounds(p:Point):
        x_ok = -0.5*0 + 0 <= p.x and p.x < maxP.x + 0.5*0
        y_ok = -0.5*0 + 0 <= p.y and p.y < maxP.y + 0.5*0
        return x_ok and y_ok
    ps = []
    for factor in range(-int(max_length)-1,int(max_length)+2,1):
        p = a.plus(uv.times(factor))
        if not in_bounds(p): continue
        ps.append(p)
#        ps.append(Point(round(p.x),round(p.y)))
    return ps

In [ ]:
points_on_ray(Point(0,0),Point(3,1),(Point(0,0),Point(11,11)))

[Point(x=0.0, y=0.0),
 Point(x=0.9486832980505138, y=0.31622776601683794),
 Point(x=1.8973665961010275, y=0.6324555320336759),
 Point(x=2.846049894151541, y=0.9486832980505138),
 Point(x=3.794733192202055, y=1.2649110640673518),
 Point(x=4.743416490252569, y=1.5811388300841898),
 Point(x=5.692099788303082, y=1.8973665961010275),
 Point(x=6.640783086353596, y=2.2135943621178655),
 Point(x=7.58946638440411, y=2.5298221281347035),
 Point(x=8.538149682454623, y=2.8460498941515415),
 Point(x=9.486832980505138, y=3.1622776601683795),
 Point(x=10.435516278555651, y=3.4785054261852175)]

**AI Prompt**

How do I round a float to the nearest integer boundary in python

**AI Response**

In Python, you can round a float to the nearest integer in two main ways:

1. Using the built-in `round()` function (not math.round):
```python
round(3.7)  # returns 4
```

2. Using `int()` with rounding:
```python
int(3.7 + 0.5)  # returns 4
```

Which method would you like me to explain further?

In [ ]:
sample_rays = """##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##"""

In [ ]:
print(sample)

............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


In [ ]:
print(sample_rays)

##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##


On further inspection of the example input and example output I think their description of the rule is inconsistent with their fiture.

They say antinodes occur "at any position exactly in line ... regardless of distance" but I notice the antinodes are spaced in a way affected by the spacing of the antennas. Also, I notice the antindoes do not always appear on the antennas generating them, despire that a distance of 0 would produce this effect.

So it seems to me that they intend "exactly in line" to have a specific meaning. Perhaps they mean that antinodes appear at the grid positions produced by reproducing the exact integer multiples that separate the original antennas. I wwill proceed on that basis and see if I can reproduce the sample, using their figure rather than their words as the truth.

In [ ]:
def points_on_ray(a:Point,b:Point,maxP):
    def in_bounds(p:Point):
        x_ok = 0 <= p.x and p.x <= maxP.x
        y_ok = 0 <= p.y and p.y <= maxP.y
        return x_ok and y_ok
    deltaP = b.minus(a)
    steps = range(0,max(maxP.x,maxP.y)+1)
    points_from_b = [b.plus(deltaP.times(step)) for step in steps]
    points_from_a = [a.plus(deltaP.times(-1 * step)) for step in steps]
    ray = [p for p in points_from_b + points_from_a if in_bounds(p)]
    return ray

In [ ]:
points_on_ray(Point(8,8),Point(6,5),Point(11,11))

[Point(x=4.0, y=2.0), Point(x=10.0, y=11.0)]

In [ ]:
points_on_ray(Point(8,8),Point(7,7),Point(11,11))

[Point(x=6.0, y=6.0),
 Point(x=5.0, y=5.0),
 Point(x=4.0, y=4.0),
 Point(x=3.0, y=3.0),
 Point(x=2.0, y=2.0),
 Point(x=1.0, y=1.0),
 Point(x=0.0, y=0.0),
 Point(x=9.0, y=9.0),
 Point(x=10.0, y=10.0),
 Point(x=11.0, y=11.0)]

This tracks the sample

In [ ]:
# redefining
def all_antinodes(d:dict[str,list[Point]],maxCorner:Point|None = None):
    antinodes = []
    for k,points in d.items():
        antinodes.extend(antinodes_for_points(points,maxCorner))
    return antinodes

In [ ]:
# redefining
def antinodes_for_points(points:list[Point],maxCorner:Point|None = None):
    for (p,q) in all_pairs(points):
        for an in antinodes_for_(p,q,maxCorner):
            yield an

In [ ]:
# redefining
def antinodes_for_(a,b,maxCorner:Point|None = None):
    if maxCorner is None:
        ba = b.times(2).minus(a)
        ab = a.times(2).minus(b)
        return (ab,ba)
    else:
        return points_on_ray(a,b,maxCorner)

In [ ]:
# redefining
def count_antinodes_on_(mmap:str, harmonics=False) -> int:
    lines = mmap.splitlines()
    lenY = len(lines)
    lenX = len(lines[0])
    def in_bounds(p:Point):
        x_ok = 0 <= p.x and p.x < lenX
        y_ok = 0 <= p.y and p.y < lenY
        return x_ok and y_ok
    maxCorner = Point(lenX-1,lenY-1) if harmonics else None
    ns = all_antinodes( parse_map( mmap ), maxCorner)
    ns_set = set([n for n in ns if in_bounds(n)])
    return len(ns_set)

In [ ]:
count_antinodes_on_(sample,harmonics=False)

14

In [ ]:
count_antinodes_on_(sample,harmonics=True)

34

**AI Prompt**

Can you see any error in my code above, which might be causing too low an output?

**AI Response**

Looking at your code, I notice a few potential issues:

1. In `points_on_ray`, you're not including the original points `a` and `b` themselves in the ray list - only the points beyond them. According to the new rules, the antenna positions themselves should be included if they're collinear.

2. The `in_bounds` check might be too strict - you might want to include points exactly on the boundary (using <= instead of <).

Would you like me to explain either of these points in more detail?

**AI Prompt**

Changing my points on ray to incldue the antennas fixed it, as you describe. But I confess I do not understand the reasoning here. If I have two antennas, and they are not exactly on top of each other, then they define a line. So when could they not be collinear?

**AI Response**

Two points always define a unique line, you're absolutely right! I should have been more precise:

The issue isn't about whether the two antennas are collinear (they always are), but rather whether any additional antennas of the same frequency lie exactly on that same line. When we include the original points in the ray calculation, we're able to detect when a third (or fourth, etc.) antenna happens to lie exactly on the line defined by any pair of antennas.

Would you like me to explain why this matters for the antinode calculation?

In [ ]:
count_antinodes_on_(inp,harmonics=True)

1045

In [ ]:
points_on_ray(Point(8,8),Point(9,9),Point(11,11))

[Point(x=9.0, y=9.0),
 Point(x=10.0, y=10.0),
 Point(x=11.0, y=11.0),
 Point(x=8.0, y=8.0),
 Point(x=7.0, y=7.0),
 Point(x=6.0, y=6.0),
 Point(x=5.0, y=5.0),
 Point(x=4.0, y=4.0),
 Point(x=3.0, y=3.0),
 Point(x=2.0, y=2.0),
 Point(x=1.0, y=1.0),
 Point(x=0.0, y=0.0)]

**AI Prompt**

Certainly. I'm particularly confused by the why, in the sample, there are antennas in points (8,8) and (9,9), and in the sample output there are NO antinodes there, but in the code which gives a correct output there ARE antinodes there. Please focus on this case.

**AI Response**

Let me look at this specific case carefully. 

In the sample output:
```
........A... (8,8)
....#....A.. (9,9)
```

You're right - this is puzzling. We have two 'A' frequency antennas at (8,8) and (9,9), and our code includes these points as antinodes (as seen in your points_on_ray output), but the sample shows a '#' between them rather than at their positions.

This seems to contradict the problem statement which says "an antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance" and "some of the new antinodes will occur at the position of each antenna".

Would you like me to look more closely at the other A-frequency antennas in the sample to see if we can spot a pattern that explains this discrepancy?

**AI Prompt**

yes. please understand this as well as you can. do not be overly didactic in your explanation.

**AI Response**

Looking at all A-frequency antennas in the sample:
```
.#...#A....# (6,5)
........A... (8,8)
....#....A.. (9,9)
```

Hmm. After checking all the antinodes shown with '#', I notice that they appear to be marking the grid points that are integer multiples of the vector difference between any pair of A antennas. The diagram shows these extensions but, contrary to the problem description, doesn't mark the antenna positions themselves as antinodes.

Yet our code including the antenna positions gives the correct answer (34). This suggests the written description is correct (include antenna positions) while the visual example is incomplete (doesn't show all antinodes).

Would you like me to examine any other patterns or aspects of this?